# 6 prepare remote deploy

## prepare turtlebot3
You can use either turtlebot3_simulator (A.) or actual turtlebot3 robot (B.).

### A. use "turtlebot3 simulator" on Ubuntu desktop

#### prepare Ubuntu desktop as a turlebot3 simulator 

1. prepare Ubuntu 16.04 desktop
    * If you use a virtual machine on VirtualBox as Ubuntu desktop, confirm below:
        * "3d acceralation" of display is "OFF"
        * set `export LIBGL_ALWAYS_SOFTWARE=1` to your .bashrc
1. set `export TURTLEBOT3_MODEL=waffle` to your .bashrc
1. install `ros-kinetic-desktop-full` and `ros-kinetic-rqt-*` using `apt`.
1. create ROS workspace.
1. clone repositories of turtlebot3_simulator from github.
    * https://github.com/ROBOTIS-GIT/turtlebot3.git
    * https://github.com/ROBOTIS-GIT/turtlebot3_msgs.git
    * https://github.com/ROBOTIS-GIT/turtlebot3_simulations.git
1. make repositories using `catkin_make`.

### B. use actual "turtlebot3" robot

#### prepare Ubuntu desktop as a turlebot3 simulator

1. prepare turtlebot3
1. make repositories using `catkin_make`.

change ${PJ_ROOT} to your directory.

In [ ]:
export PJ_ROOT="${HOME}/roboticbase-core"
cd ${PJ_ROOT};pwd

example)
```
/Users/user/roboticbase-core
```

## load environment variables

In [ ]:
source ${PJ_ROOT}/docs/minikube/env

## set external ip addr of minikube machine

In [ ]:
export EXTERNAL_HOST_IPADDR=$(ifconfig en0 | awk '/inet / {print $2}');echo ${EXTERNAL_HOST_IPADDR}

## setup docker

1. install required packages

```bash
turtlebot3@turtlebot3:~$ sudo apt update
turtlebot3@turtlebot3:~$ sudo apt update -y
turtlebot3@turtlebot3:~$ sudo apt install apt-transport-https ca-certificates curl software-properties-common
  ```
2. add Docker’s official GPG key:

```bash
turtlebot3@turtlebot3:~$ curl -fsSL https://download.docker.com/linux/ubuntu/gpg | sudo apt-key add -
```
3. set up the stable repository of Docker

```bash
turtlebot3@turtlebot3:~$ sudo add-apt-repository "deb [arch=amd64] https://download.docker.com/linux/ubuntu $(lsb_release -cs) stable"
```
4. install Docker

```bash
turtlebot3@turtlebot3:~$ sudo apt update
turtlebot3@turtlebot3:~$ sudo apt install docker-ce -y
```
5. verify instration

```bash
turtlebot3@turtlebot3:~$ sudo docker run hello-world
```

### add inserucre registry

In [ ]:
echo "sudo mkdir -p /etc/systemd/system/docker.service.d/; cat << __EOS__ | sudo tee /etc/systemd/system/docker.service.d/override.conf
[Service]
ExecStart=
ExecStart=/usr/bin/dockerd -H fd:// --insecure-registry=${EXTERNAL_HOST_IPADDR}:5000
__EOS__"

1. add insecure registry to turtlebot3's docker using above command
2. restart docker daemon

```bash
turtlebot3@turtlebot3-fake:~$ sudo systemctl daemon-reload
turtlebot3@turtlebot3-fake:~$ sudo systemctl restart docker.service
turtlebot3@turtlebot3-fake:~$ sudo systemctl status docker.service
```

## setup minikube

1. setup minikube

```bash
turtlebot3@turtlebot3:~$ curl -Lo minikube https://storage.googleapis.com/minikube/releases/latest/minikube-linux-amd64 && chmod +x minikube && sudo cp minikube /usr/local/bin/ && rm minikube
```
2. verify instration

```bash
turtlebot3@turtlebot3:~$ minikube version
```

## setup kubectl

1. setup kubectl

```bash
turtlebot3@turtlebot3:~$ curl -Lo kubectl https://storage.googleapis.com/kubernetes-release/release/$(curl -s https://storage.googleapis.com/kubernetes-release/release/stable.txt)/bin/linux/amd64/kubectl && chmod +x kubectl && sudo cp kubectl /usr/local/bin/ && rm kubectl
```
2. verify instration

```bash
turtlebot3@turtlebot3:~$ kubectl version --client
```

## start minikube without virtualization

### generate a command to add Insecure Repository

In [ ]:
NWNAME=$(VBoxManage showvminfo minikube | grep "Host-only Interface" | awk 'match($0, /vboxnet[0-9]+/){print substr($0,RSTART,RLENGTH)}')
HOST_IPADDR=$(ifconfig ${NWNAME} | awk '/inet / {print $2}')
NETMASK_HEX=$(ifconfig ${NWNAME} | awk '/netmask / {print $4}')
NETMASK=$(echo "${NETMASK_HEX:2}" | perl -pe '$_ = unpack("B32", pack("H*", $_)); s/0+$//g; $_ = length')
echo 'cat ${HOME}/.minikube/machines/minikube/config.json | perl -pse '"'"'s/"InsecureRegistry": \[/"InsecureRegistry": [\n                "$h\/$m",/g;'"' -- -h=${EXTERNAL_HOST_IPADDR} -m=${NETMASK}"' > /tmp/config.json;mv /tmp/config.json ${HOME}/.minikube/machines/minikube/config.json'

### start minikube

1. delete minikube if already started

```bash
turtlebot3@turtlebot3:~$ sudo minikube stop
turtlebot3@turtlebot3:~$ sudo minikube delete
```
2. create `.kube/config`

```bash
turtlebot3@turtlebot3:~$ rm -rf $HOME/.kube/
turtlebot3@turtlebot3:~$ mkdir -p $HOME/.kube
turtlebot3@turtlebot3:~$ touch $HOME/.kube/config
```
3. setup environment variables

```bash
turtlebot3@turtlebot3:~$ export MINIKUBE_HOME=$HOME
turtlebot3@turtlebot3:~$ export CHANGE_MINIKUBE_NONE_USER=true
turtlebot3@turtlebot3:~$ export KUBECONFIG=$HOME/.kube/config
```
4. start minikube without virtualization

```bash
turtlebot3@turtlebot3:~$ sudo -E minikube start --memory 2048 --vm-driver=none
```
5. rewite `${HOME}/.minikube/machines/minikube/config.json` using above command
6. restart minikube

```bash
turtlebot3@turtlebot3-fake:~$ minikube stop
turtlebot3@turtlebot3-fake:~$ sudo -E minikube start --memory 2048
```
7. verify minikube

```bash
turtlebot3@turtlebot3:~$ kubectl version
turtlebot3@turtlebot3:~$ kubectl get nodes
```

### confirm the dns settings of minkube

1. try to dig `www.google.com`

```bash
turtlebot3@turtlebot3:~$ kubectl run -it --rm --restart=Never dig --image tutum/dnsutils -- dig www.google.com
```
2. if you encountered `connection timed out`, you have to setup the additional nameservers to `kube-dns`

### setup additional nameservers to `kube-dns`

1. prepare `/tmp/kube-dns-configmap.yaml` like below:

```yaml
apiVersion: v1
kind: ConfigMap
metadata:
  name: kube-dns
  namespace: kube-system
  labels:
    addonmanager.kubernetes.io/mode: EnsureExists
data:
  upstreamNameservers: |-
    ["8.8.8.8", "8.8.4.4"]
```
2. apply `/tmp/kube-dns-configmap.yaml`

```bash
turtlebot3@turtlebot3:~$ kubectl apply -f /tmp/kube-dns-configmap.yaml
```
3. delete the pod of `kube-dns` (restart `kube-dns` automatically)

```bash
turtlebot3@turtlebot3:~$ kubectl delete pod -n kube-system $(kubectl get pods -n kube-system -l k8s-app=kube-dns -o template --template "{{(index .items 0).metadata.name}}")
```
4. confirm that `kube-dns` is started

```bash
turtlebot3@turtlebot3:~$ kubectl get pods -n kube-system -l k8s-app=kube-dns
NAME                        READY     STATUS    RESTARTS   AGE
kube-dns-86f4d74b45-x7m75   3/3       Running   0          2m
```
5. retry to dig `www.google.com`

```bash
turtlebot3@turtlebot3:~$ kubectl run -it --rm --restart=Never dig --image tutum/dnsutils -- dig www.google.com
```

## deploy `deployer` to turtlebot3

### prepare a command to create Secret of username & password

In [ ]:
echo "kubectl create secret generic mqtt-username-password --from-literal=mqtt_username=ros --from-literal=mqtt_password=${MQTT__ros}"

1. create Secret of username & password of MQTT Broker using above command

### prepare a command to create ConfigMap of mqtt endpoint

In [ ]:
echo "kubectl create configmap mqtt-config --from-literal=mqtt_use_tls=false --from-literal=mqtt_host=${EXTERNAL_HOST_IPADDR} --from-literal=mqtt_port=1883 --from-literal=mqtt_cmd_topic=/${DEPLOYER_TYPE}/${DEPLOYER_ID}"

1. create ConfigMap of host & port & topic to connect MQTT Broker using above command

### start `deployer` to operate k8s Resources through MQTT

1. prepare `/tmp/mqtt-kube-operator.yaml` like below:

```yaml
apiVersion: v1
kind: ServiceAccount
metadata:
  name: mqtt-kube-operator
---
apiVersion: rbac.authorization.k8s.io/v1
kind: Role
metadata:
  name: mqtt-kube-operator
  namespace: default
rules:
- apiGroups: [""]
  resources: ["services", "configmaps", "secrets"]
  verbs: ["get", "list", "create", "update", "delete"]
- apiGroups: ["apps"]
  resources: ["deployments"]
  verbs: ["get", "list", "create", "update", "delete"]
---
apiVersion: rbac.authorization.k8s.io/v1
kind: RoleBinding
metadata:
  name: mqtt-kube-operator
  namespace: default
roleRef:
  apiGroup: rbac.authorization.k8s.io
  kind: Role
  name: mqtt-kube-operator
subjects:
- kind: ServiceAccount
  name: mqtt-kube-operator
  namespace: default
---
apiVersion: apps/v1
kind: Deployment
metadata:
  name: mqtt-kube-operator
spec:
  replicas: 1
  selector:
    matchLabels:
      app: mqtt-kube-operator
  template:
    metadata:
      labels:
        app: mqtt-kube-operator
    spec:
      serviceAccountName: mqtt-kube-operator
      containers:
      - name: mqtt-kube-operator
        image: techsketch/mqtt-kube-operator:0.1.0
        imagePullPolicy: Always
        env:
        - name: MQTT_USERNAME
          valueFrom:
            secretKeyRef:
              name: mqtt-username-password
              key: mqtt_username
        - name: MQTT_PASSWORD
          valueFrom:
            secretKeyRef:
              name: mqtt-username-password
              key: mqtt_password
        - name: MQTT_USE_TLS
          valueFrom:
            configMapKeyRef:
              name: mqtt-config
              key: mqtt_use_tls
        - name: MQTT_HOST
          valueFrom:
            configMapKeyRef:
              name: mqtt-config
              key: mqtt_host
        - name: MQTT_PORT
          valueFrom:
            configMapKeyRef:
              name: mqtt-config
              key: mqtt_port
        - name: MQTT_CMD_TOPIC
          valueFrom:
            configMapKeyRef:
              name: mqtt-config
              key: mqtt_cmd_topic
```
2. apply `/tmp/mqtt-kube-operator.yaml`

```bash
turtlebot3@turtlebot3:~$ kubectl apply -f /tmp/mqtt-kube-operator.yaml
```
3. confirm that `mqtt-kube-operator` connect to MQTT Broker

```bash
turtlebot3@turtlebot3:~$ kubectl logs -f $(kubectl get pods -l app=mqtt-kube-operator -o template --template "{{(index .items 0).metadata.name}}")
```

## register deployer service

In [ ]:
TOKEN=$(cat ${PJ_ROOT}/secrets/auth-tokens.json | jq '.bearer_tokens[0].token' -r)
curl -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-ServicePath: ${DEPLOYER_SERVICEPATH}" -H "Content-Type: application/json" http://${HOST_IPADDR}:8080/idas/ul20/manage/iot/services/ -X POST -d @- <<__EOS__
{
  "services": [
    {
      "apikey": "${DEPLOYER_TYPE}",
      "cbroker": "http://orion:1026",
      "resource": "/iot/d",
      "entity_type": "${DEPLOYER_TYPE}"
    }
  ]
}
__EOS__

expected)
```json
{}
```

### confirm registered service

In [ ]:
TOKEN=$(cat ${PJ_ROOT}/secrets/auth-tokens.json | jq '.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${DEPLOYER_SERVICEPATH}" http://${HOST_IPADDR}:8080/idas/ul20/manage/iot/services/ | jq .

example)
```bash
{
  "count": 1,
  "services": [
    {
      "_id": "5b90ba6c0c4a497125314e7c",
      "subservice": "/deployer",
      "service": "fiwaredemo",
      "apikey": "deployer",
      "resource": "/iot/d",
      "__v": 0,
      "attributes": [],
      "lazy": [],
      "commands": [],
      "entity_type": "deployer",
      "internal_attributes": [],
      "static_attributes": []
    }
  ]
}
```

## register deployer device

In [ ]:
TOKEN=$(cat ${PJ_ROOT}/secrets/auth-tokens.json | jq '.bearer_tokens[0].token' -r)
curl -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-ServicePath: ${DEPLOYER_SERVICEPATH}" -H "Content-Type: application/json" http://${HOST_IPADDR}:8080/idas/ul20/manage/iot/devices/ -X POST -d @- <<__EOS__
{
  "devices": [
    {
      "device_id": "${DEPLOYER_ID}",
      "entity_name": "${DEPLOYER_ID}",
      "entity_type": "${DEPLOYER_TYPE}",
      "timezone": "Asia/Tokyo",
      "protocol": "UL20",
      "attributes": [],
      "commands": [
        {
          "name": "apply",
          "type": "string"
        }, {
          "name": "delete",
          "type": "string"
        }
      ],
      "transport": "AMQP"
    }
  ]
}
__EOS__

expected)
```json
{}
```

### confirm registered device

In [ ]:
TOKEN=$(cat ${PJ_ROOT}/secrets/auth-tokens.json | jq '.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${DEPLOYER_SERVICEPATH}" http://${HOST_IPADDR}:8080/idas/ul20/manage/iot/devices/${DEPLOYER_ID}/ | jq .

example)
```bash
{
  "device_id": "deployer_01",
  "service": "fiwaredemo",
  "service_path": "/deployer",
  "entity_name": "deployer_01",
  "entity_type": "deployer",
  "transport": "AMQP",
  "attributes": [],
  "lazy": [],
  "commands": [
    {
      "object_id": "apply",
      "name": "apply",
      "type": "string"
    },
    {
      "object_id": "delete",
      "name": "delete",
      "type": "string"
    }
  ],
  "static_attributes": [],
  "protocol": "UL20"
}
```

In [ ]:
TOKEN=$(cat ${PJ_ROOT}/secrets/auth-tokens.json | jq '.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${DEPLOYER_SERVICEPATH}" http://${HOST_IPADDR}:8080/orion/v2/entities/${DEPLOYER_ID}/ | jq .

example)
```bash
{
  "id": "deployer_01",
  "type": "deployer",
  "TimeInstant": {
    "type": "ISO8601",
    "value": " ",
    "metadata": {}
  },
  "apply_info": {
    "type": "commandResult",
    "value": " ",
    "metadata": {}
  },
  "apply_status": {
    "type": "commandStatus",
    "value": "UNKNOWN",
    "metadata": {}
  },
  "delete_info": {
    "type": "commandResult",
    "value": " ",
    "metadata": {}
  },
  "delete_status": {
    "type": "commandStatus",
    "value": "UNKNOWN",
    "metadata": {}
  },
  "apply": {
    "type": "string",
    "value": "",
    "metadata": {}
  },
  "delete": {
    "type": "string",
    "value": "",
    "metadata": {}
  }
}
```

## test publishing the `apply` command of `deployer`

### prepare a command to emulate sending command

In [ ]:
TOKEN=$(cat ${PJ_ROOT}/secrets/auth-tokens.json | jq '.bearer_tokens[0].token' -r)
echo -e "curl -i -H \"Authorization: bearer ${TOKEN}\" -H \"Fiware-Service: ${FIWARE_SERVICE}\" -H \"Fiware-Servicepath: ${DEPLOYER_SERVICEPATH}\" -H \"Content-Type: application/json\" http://${HOST_IPADDR}:8080/orion/v1/updateContext -d @-<<__EOS__
{
  \"contextElements\": [
    {
      \"id\": \"${DEPLOYER_ID}\",
      \"isPattern\": \"false\",
      \"type\": \"${DEPLOYER_TYPE}\",
      \"attributes\": [
        {
          \"name\": \"apply\",
          \"value\": \"{}\"
        }
      ]
    }
  ],
  \"updateAction\": \"UPDATE\"
}
__EOS__"

### subscribe all topics

In [ ]:
mosquitto_sub -h ${HOST_IPADDR} -p 1883 -d -u iotagent -P ${MQTT__iotagent} -t /#

### send message to deployer endpoint

_Outside of this notebook_
1. open a ternminal.
1. run a command displayed `prepare a command to emulate sending command`.
1. stop the `subscribe all topics` cell.

when executing the command, show below message on subscriber cell.
example)
```
Client mosqsub|70811-MacBook-P received PUBLISH (d0, q0, r0, m0, '/deployer/deployer_01/cmd', ... (20 bytes))
deployer_01@apply|{}
Client mosqsub|70811-MacBook-P received PUBLISH (d0, q0, r0, m0, '/deployer/deployer_01/cmdexe', ... (50 bytes))
deployer_01@apply|ignore format, skip this message
```

### confirm `deployer` log

example)

```
2018-09-06T23:34:17.369Z	INFO	handlers/messageHandler.go:76	received message: "deployer_01@apply|{}"
2018-09-06T23:34:17.369Z	INFO	handlers/messageHandler.go:102	data: "{}"
2018-09-06T23:34:17.369Z	INFO	handlers/messageHandler.go:133	ignore format, skip this message: Object 'Kind' is missing in '{}'
2018-09-06T23:34:17.369Z	INFO	handlers/messageHandler.go:85	send message: "deployer_01@apply|ignore format, skip this message"
```

### confirm deployer entity

In [ ]:
TOKEN=$(cat ${PJ_ROOT}/secrets/auth-tokens.json | jq '.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${DEPLOYER_SERVICEPATH}" http://${HOST_IPADDR}:8080/orion/v2/entities/${DEPLOYER_ID}/ | jq .

example)
```bash
{
  "id": "deployer_01",
  "type": "deployer",
  "TimeInstant": {
    "type": "ISO8601",
    "value": "2018-09-06T23:34:17.00Z",
    "metadata": {}
  },
  "apply_info": {
    "type": "commandResult",
    "value": "ignore format, skip this message",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2018-09-06T23:34:17.357Z"
      }
    }
  },
  "apply_status": {
    "type": "commandStatus",
    "value": "OK",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2018-09-06T23:34:17.357Z"
      }
    }
  },
  "delete_info": {
    "type": "commandResult",
    "value": " ",
    "metadata": {}
  },
  "delete_status": {
    "type": "commandStatus",
    "value": "UNKNOWN",
    "metadata": {}
  },
  "apply": {
    "type": "string",
    "value": "",
    "metadata": {}
  },
  "delete": {
    "type": "string",
    "value": "",
    "metadata": {}
  }
}
```